In [2]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import random
import datetime
import re
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
import pyttsx3
import speech_recognition as sr
import pywhatkit
import wikipedia
import webbrowser
import pyjokes
import os

In [3]:

# ===============================
# BLOCK 2: INITIALIZE TEXT-TO-SPEECH
# ===============================
engine = pyttsx3.init()

def speak(text):
    """Convert text to speech."""
    engine.say(text)
    engine.runAndWait()

In [4]:
# ===============================
# BLOCK 3: VOICE INPUT FUNCTION
# ===============================
def take_command():
    """Listen to user via microphone and return text."""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
    try:
        query = r.recognize_google(audio)
        print("You said:", query)
        return query.lower()
    except sr.UnknownValueError:
        speak("Sorry, I didn't catch that.")
        return ""
    except sr.RequestError:
        speak("Sorry, I am unable to access Google Speech Recognition.")
        return ""


In [5]:
# ===============================
# BLOCK 4: GREETING FUNCTION
# ===============================
def greet():
    """Greet user based on current time."""
    hour = datetime.datetime.now().hour
    if hour < 12:
        speak("Good morning!")
    elif hour < 18:
        speak("Good afternoon!")
    else:
        speak("Good evening!")
    speak("I am your AI assistant. How can I help you today?")

In [6]:
# ===============================
# BLOCK 5: LOAD INTENTS DATA & PREP NLP/ML MODEL
# ===============================
# Load CSV
data = pd.read_csv(r'C:\Users\Devanshhh\Desktop\DEV_AI_MODEL\model_intents.csv')  # Make sure intents.csv is in same folder

# Preprocessing function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

X = data['patterns'].apply(clean_text)
y = data['intent']

# Vectorize text
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)

# Train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_vec, y_enc)
print("✅ NLP ML Model trained successfully!")

# Function to predict intent
def get_intent(user_input):
    user_input_clean = clean_text(user_input)
    user_vec = vectorizer.transform([user_input_clean])
    probs = model.predict_proba(user_vec)
    intent_idx = np.argmax(probs)
    intent = le.inverse_transform([intent_idx])[0]
    confidence = np.max(probs)
    if confidence < 0.3:  # Low confidence fallback
        return "unknown"
    return intent


✅ NLP ML Model trained successfully!


In [7]:
# ===============================
# BLOCK 6: WEATHER FUNCTION
# ===============================
def get_weather(city):
    api_key = "1a0b53c75d3d5444eaa2c945b1cd0b6e"  # Your API key
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        res = requests.get(url)
        data = res.json()
        if data.get("cod") != 200:
            return "City not found."
        temp = data['main']['temp']
        desc = data['weather'][0]['description']
        return f"Current temperature in {city.title()} is {temp}°C with {desc}."
    except:
        return "Unable to fetch weather at the moment."



In [8]:
# ===============================
# BLOCK 7: WHATSAPP FUNCTION
# ===============================
contacts = {
     "mom": "+91993099795",
    "dad": "+919920578951",
    "devansh": "+919930993699",
    "siddharth": "+918010340539",
    "appu": "+918828216775",
    "shubham": "+918369246841",
    "dweep": "+917710978922",
    "ria": "+919082432506",
    "srishti": "+919820125493",
    "shiv": "+918530496292",
    "aneesh": "+918767688040",
    "ayush": "+918104722563",
    "kk": "+919702479662",
    "nitin": "+919022589784",
    "mama": "+919324024818"
    # Add more as needed
}

def send_whatsapp(name, msg):
    """Send WhatsApp message if contact exists."""
    if name in contacts:
        pywhatkit.sendwhatmsg_instantly(contacts[name], msg)
        return f"Message sent to {name}."
    else:
        return "Contact not found."


In [9]:

# ===============================
# BLOCK 8: RESPONSE HANDLER
# ===============================
def get_response(user_input):
    """Return response string based on keywords or ML intent."""
    lower_input = user_input.lower()

    # ================= Keyword triggers =================
    if "time" in lower_input:
        return f"The current time is {datetime.datetime.now().strftime('%H:%M:%S')}."
    elif "date" in lower_input:
        return f"Today's date is {datetime.datetime.now().strftime('%Y-%m-%d')}."
    elif "joke" in lower_input:
        return pyjokes.get_joke()
    elif "youtube" in lower_input:
        webbrowser.open("https://www.youtube.com")
        return "Opening YouTube."
    elif "google" in lower_input or "search" in lower_input:
        query = lower_input.replace("search", "").replace("google", "")
        webbrowser.open(f"https://www.google.com/search?q={query.strip()}")
        return f"Showing Google results for {query.strip()}."
    elif "wikipedia" in lower_input:
        query = lower_input.replace("wikipedia", "")
        try:
            result = wikipedia.summary(query, sentences=2)
            return "According to Wikipedia: " + result
        except:
            return "Sorry, could not fetch Wikipedia info."
    elif "send whatsapp" in lower_input:
        speak("Whom should I message?")
        name = take_command()
        speak("What is the message?")
        msg = take_command()
        return send_whatsapp(name, msg)
    elif "weather" in lower_input:
        parts = lower_input.split("in")
        if len(parts) > 1:
            city = parts[1].strip()
        else:
            speak("Which city?")
            city = take_command()
        return get_weather(city)

    # ================= ML fallback for conversation =================
    intent = get_intent(user_input)
    if intent == "unknown":
        return "Sorry, I am still learning. Can you rephrase that?"

    possible_responses = data[data['intent'] == intent]['response'].values
    if len(possible_responses) > 0:
        return random.choice(possible_responses)
    return "Hmm, I’m not sure about that."



In [10]:
# ===============================
# BLOCK 9: CONTINUOUS VOICE ASSISTANT LOOP
# ===============================
def chat_mode_voice():
    """Continuous assistant listening until exit."""
    greet()
    print("Voice Assistant Ready! Say 'exit' to stop.\n")
    
    while True:
        user_input = take_command()
        if user_input.lower() in ["exit", "quit", "bye"]:
            speak("Goodbye! Take care.")
            print("Assistant: Goodbye! Take care 👋")
            break
        
        response = get_response(user_input)
        print("Assistant:", response)
        speak(response)



In [11]:
# ===============================
# BLOCK 10: RUN ASSISTANT
# ===============================
chat_mode_voice()

Voice Assistant Ready! Say 'exit' to stop.

Listening...
You said: what is the time
Assistant: The current time is 12:30:44.
Listening...
You said: thank u bye
Assistant: Sorry, I am still learning. Can you rephrase that?
Listening...
You said: thank
Assistant: Sorry, I am still learning. Can you rephrase that?
Listening...
Assistant: Sorry, I am still learning. Can you rephrase that?
Listening...
Assistant: Sorry, I am still learning. Can you rephrase that?
Listening...
You said: thank
Assistant: Sorry, I am still learning. Can you rephrase that?
Listening...
You said: exit
Assistant: Goodbye! Take care 👋
